In [ ]:
import os
import sys  
import json
import torch 
import argparse
import numpy as np
from PIL import Image 
from tqdm import tqdm 
from utils import model_gen, load_jsonl
from transformers import AutoModelForCausalLM, AutoTokenizer  
  
ckpt_path = 'internlm/internlm-xcomposer2-vl-7b'
tokenizer = AutoTokenizer.from_pretrained(ckpt_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(ckpt_path, device_map="cuda", trust_remote_code=True).eval().cuda().half()
model.tokenizer = tokenizer

In [ ]:
samples = load_jsonl('./data/ai2d_test.jsonl')
image_root = './data/ai2d/'
import re
pattern = re.compile(r'[A-Z]')

results = []
for sample in tqdm(samples):
    text = sample['text']
    image = os.path.join(image_root, sample['image'])
    with torch.cuda.amp.autocast():
        response = model_gen(model, text, image) 
    res = pattern.findall(response)
    if len(res) == 0:
        print('Error:', output_text); res = 'E'
    results.append(res[0] == sample['answer'])
print (np.mean(results))